> scikit-learn transformer for conducting K-Means clustering to get neighbourhoods from latitude and longitude

* Transformer sets a number of "neighbourhoods" and a seed on initiation
* During fitting an unsupervised k-means clustering model is fit
* Upon transformation, the neighbourhoods (as integers) are returned
* The transformed data is returned as a column vector for easier use with other transformers

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.cluster import KMeans

In [ ]:
class NeighbourhoodCluster(BaseEstimator, TransformerMixin):
    def __init__(self, num_clusters, seed):
        self.num_clusters = num_clusters
        self.seed = seed
        self.cluster = KMeans(self.num_clusters, random_state=self.seed)
        self.keys = ['latitude', 'longitude']

    def fit(self, X, y=None, **args):
        # Better method of removing outliers can be found in the notebook
        # https://www.kaggle.com/arnaldcat/two-sigma-connect-rental-listing-inquiries/unsupervised-and-supervised-neighborhood-encoding
        mask = (
            (X["longitude"] < -73.75) & 
            (X["longitude"] > -74.05) & 
            (X["latitude"] > 40.4) & 
            (X["latitude"] < 40.9)
        )
        self.cluster.fit(X[mask][self.keys])
        return self

    def transform(self, X, y=None, **args):
        clusters = self.cluster.predict(X[self.keys])
        return clusters[:, None]

    def get_feature_names(self):
        return ['neighbourhood']

In [ ]:
# Load in train and test data
train = pd.read_json('../input/train.json')
test = pd.read_json('../input/test.json')

train['train'] = 1
test['train'] = 0

X = pd.concat([train, test])
y = X.pop('interest_level')

In [ ]:
num_neighbourhoods = 10
seed = 42
transformer = NeighbourhoodCluster(num_neighbourhoods, seed)

In [ ]:
neighbourhoods = transformer.fit_transform(X)

In [ ]:
neighbourhoods[:10]

Depending on the classifier being used, it might be worth it to encode the values into separate columns (i.e. one hot encoding). In such a scenario, the user of sklearn pipelines would be a good place to start.